In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from io import StringIO

In [2]:
csv_data = '''A,B,C,D
    1.0,2.0,3.0,4.0
    5.0,6.0,,7.0
    10.0,20.0,11.0,'''

In [3]:
df = pd.read_csv(StringIO(csv_data))
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,7.0
2,10.0,20.0,11.0,NaN


In [4]:
#Eleminnating training examples with missing values
df.dropna(axis = 0) # removing rows

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [5]:
df.dropna(axis = 1) #Removving Columns

,A,B
0,1.0,2.0
1,5.0,6.0
2,10.0,20.0


In [6]:
#only drop the rows where all columns are nan
df.dropna(how='all')

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,7.0
2,10.0,20.0,11.0,NaN


Removing the rows or columns may seem convinient but in many cases it eads to loss of information, for example, we may remove to many rows such that our model may not be able to learn from the dataset very well, or we can delete too many columns such that our model loses the critical feature set of the dataset

Alternative is to interpolate the data, the simplest way is mean imputation

In [7]:
from sklearn.impute import SimpleImputer

In [8]:
imr = SimpleImputer(missing_values=np.nan, strategy='mean')
imr = imr.fit(df.values)
imputed_data = imr.transform(df.values)
imputed_data
#we have replaced the nan value with the mean which is calculted separately for each parameter

array([[ 1. ,  2. ,  3. ,  4. ],
       [ 5. ,  6. ,  7. ,  7. ],
       [10. , 20. , 11. ,  5.5]])

In [9]:
#Alternatively
df.fillna(df.mean())

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,7.0,7.0
2,10.0,20.0,11.0,5.5


In [10]:
#Catagorical data encodings with pandas
df = pd.DataFrame([
['green', 'M', 10.1, 'class2'],
['red', 'L', 13.5, 'class1'],
['blue', 'XL', 15.3, 'class2']])
df.columns = ['color', 'size', 'price', 'class_labels']

In [11]:
df
#nominal features  - color
#ordinal features - size

,color,size,price,class_labels
0,green,M,10.1,class2
1,red,L,13.5,class1
2,blue,XL,15.3,class2


In [12]:
#converting ordinal features to integers so that our model interprets them conviniently
size_mapping = {
    'XL':3,
    'L':2,
    'M':1
}
df['size'] = df['size'].map(size_mapping)
df

,color,size,price,class_labels
0,green,1,10.1,class2
1,red,2,13.5,class1
2,blue,3,15.3,class2


In [13]:
#when we awnt to trasform the results back into their original form we will need reverse mapping
inv_size_mapping = {v: k for k , v in size_mapping.items()}
df['size'].map(inv_size_mapping)

0     M
1     L
2    XL
Name: size, dtype: object

In [14]:
#encoding class labels
class_mapping = {label:idx for idx,label in enumerate(np.unique(df['class_labels']))}

In [15]:
class_mapping

{'class1': 0, 'class2': 1}

In [16]:
df['class_labels'] = df['class_labels'].map(class_mapping)
df

,color,size,price,class_labels
0,green,1,10.1,1
1,red,2,13.5,0
2,blue,3,15.3,1


In [18]:
#we can also create an inverse mapping to revert back the class encoding
inv_class_map = {v:k for k,v in class_mapping.items()}

In [19]:
inv_class_map

{0: 'class1', 1: 'class2'}

In [20]:
df['class_labels'] = df['class_labels'].map(inv_class_map)

In [21]:
df

,color,size,price,class_labels
0,green,1,10.1,class2
1,red,2,13.5,class1
2,blue,3,15.3,class2


In [22]:
#Alternativel we can use sklearn libraries to encode the class variables
from sklearn.preprocessing import LabelEncoder

In [23]:
class_le = LabelEncoder()
y = class_le.fit_transform(df['class_labels'].values)
y

array([1, 0, 1])

In [24]:
#we can use inverse transform method to revert back the changes
class_le.inverse_transform(y)

array(['class2', 'class1', 'class2'], dtype=object)

In [26]:
#using label encoder on color column
X = df[['color', 'size','price']].values

In [29]:
color_le = LabelEncoder()

In [30]:
X[:,0] = color_le.fit_transform(X[:,0])
X

array([[1, 1, 10.1],
       [2, 2, 13.5],
       [0, 3, 15.3]], dtype=object)

In [32]:
from sklearn.preprocessing import OneHotEncoder

In [33]:
#using one hot encoding for color 
X = df[['color', 'size','price']].values
color_ohe = OneHotEncoder()
color_ohe.fit_transform(X[:, 0].reshape(-1, 1)).toarray()

array([[0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.]])

In [34]:
from sklearn.compose import ColumnTransformer
X = df[['color','size','price']].values

In [35]:
c_transf = ColumnTransformer([('onehot',OneHotEncoder(),[0]),
                             ('nothing','passthrough',[1,2])])

In [37]:
c_transf.fit_transform(X).astype(float)

array([[ 0. ,  1. ,  0. ,  1. , 10.1],
       [ 0. ,  0. ,  1. ,  2. , 13.5],
       [ 1. ,  0. ,  0. ,  3. , 15.3]])

In the preceding code example, we specified that we want to modify only the
first column and leave the other two columns untouched via the 'passthrough'
argument.

In [41]:
#more convinient way is use pandas method get_dummies
pd.get_dummies(df[['price','size','color']])

,price,size,color_blue,color_green,color_red
0,10.1,1,0,1,0
1,13.5,2,0,0,1
2,15.3,3,1,0,0


In [42]:
#We can simplify our dataset by dropping the redundant column 
pd.get_dummies(df[['size','price','color']],drop_first=True)

,size,price,color_green,color_red
0,1,10.1,1,0
1,2,13.5,0,1
2,3,15.3,0,0
